In [15]:
import requests
import pandas as pd
import plotly.express as px

API_KEY = "f8t2eKKxjdTt7LiuD0Na5nMB0aYdpdgi"
TICKER = "MSFT"

# Nuevo endpoint “stable” para precios EOD completos (open, high, low, close, volumen, etc.)
url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={TICKER}&apikey={API_KEY}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    print("ERROR:", resp.status_code)
    print("Respuesta del servidor:", resp.text)
    raise RuntimeError(f"Error al conectar con FMP (stable endpoint): {resp.status_code}")

data = resp.json()

# aquí puede que la estructura JSON sea distinta: por ejemplo “historical” dentro de un campo “historical” o directamente raíz
if "historical" in data:
    hist = data["historical"]
else:
    # si la respuesta es directamente un arreglo
    hist = data

df = pd.DataFrame(hist)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

fig = px.line(df, x="date", y="close",
              title=f"Precio de cierre diario — {TICKER} (endpoint stable)",
              labels={"date": "Fecha", "close": "Precio (USD)"},
              template="plotly_white")
fig.show()

print(df.head(), df.tail())


     symbol       date    open    high     low   close    volume  change  \
1254   MSFT 2020-10-16  220.15  222.29  219.32  219.66  26057900   -0.49   
1253   MSFT 2020-10-19  220.42  222.30  213.72  214.22  27625841   -6.19   
1252   MSFT 2020-10-20  215.80  217.37  213.09  214.65  22753511   -1.15   
1251   MSFT 2020-10-21  213.12  216.92  213.12  214.80  22724906    1.68   
1250   MSFT 2020-10-22  213.93  216.06  211.70  214.89  22351500    0.96   

      changePercent      vwap  
1254       -0.22258  220.3550  
1253       -2.81000  217.6650  
1252       -0.53290  215.2275  
1251        0.78829  214.4900  
1250        0.44874  214.1450     symbol       date     open    high     low   close    volume  change  \
4   MSFT 2025-10-09  522.340  524.33  517.40  522.40  18343602   0.065   
3   MSFT 2025-10-10  519.640  523.58  509.63  510.96  24133840  -8.680   
2   MSFT 2025-10-13  516.410  516.41  511.68  514.05  14284238  -2.360   
1   MSFT 2025-10-14  510.230  515.28  506.00  513.57  1

In [16]:
import requests
import pandas as pd
import plotly.express as px

TICKER = "AAPL"

url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={TICKER}&apikey={API_KEY}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    print("ERROR:", resp.status_code)
    print("Respuesta del servidor:", resp.text)
    raise RuntimeError(f"Error al conectar con FMP (stable endpoint) para {TICKER}: {resp.status_code}")

data = resp.json()
hist = data["historical"] if "historical" in data else data

df = pd.DataFrame(hist)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

fig = px.line(df, x="date", y="close",
              title=f"Precio de cierre diario — {TICKER} (endpoint stable)",
              labels={"date": "Fecha", "close": "Precio (USD)"},
              template="plotly_white")
fig.show()

print(df.head(), df.tail())


     symbol       date    open    high     low   close     volume  change  \
1254   AAPL 2020-10-16  121.28  121.55  118.81  119.02  115393808   -2.26   
1253   AAPL 2020-10-19  119.96  120.42  115.66  115.98  120639337   -3.98   
1252   AAPL 2020-10-20  116.20  118.98  115.63  117.51  124423728    1.31   
1251   AAPL 2020-10-21  116.67  118.71  116.45  116.87   89946000    0.20   
1250   AAPL 2020-10-22  117.45  118.04  114.59  115.75  101988000   -1.70   

      changePercent      vwap  
1254       -1.86000  120.1650  
1253       -3.32000  118.0050  
1252        1.13000  117.0800  
1251        0.17142  117.1750  
1250       -1.45000  116.4575     symbol       date     open    high     low   close    volume  change  \
4   AAPL 2025-10-09  257.810  258.00  253.14  254.04  38322012  -3.770   
3   AAPL 2025-10-10  254.940  256.38  244.00  245.27  61999100  -9.670   
2   AAPL 2025-10-13  249.380  249.69  245.56  247.66  38142942  -1.720   
1   AAPL 2025-10-14  246.600  248.85  244.70  247

In [17]:
import requests
import pandas as pd
import plotly.express as px

TICKER = "JNJ"

url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={TICKER}&apikey={API_KEY}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    print("ERROR:", resp.status_code)
    print("Respuesta del servidor:", resp.text)
    raise RuntimeError(f"Error al conectar con FMP (stable endpoint) para {TICKER}: {resp.status_code}")

data = resp.json()
hist = data["historical"] if "historical" in data else data

df = pd.DataFrame(hist)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

fig = px.line(df, x="date", y="close",
              title=f"Precio de cierre diario — {TICKER} (endpoint stable)",
              labels={"date": "Fecha", "close": "Precio (USD)"},
              template="plotly_white")
fig.show()

print(df.head(), df.tail())


     symbol       date    open    high     low   close   volume  change  \
1254    JNJ 2020-10-16  147.60  148.76  147.01  148.10  6098204    0.50   
1253    JNJ 2020-10-19  148.53  148.90  144.13  144.32  7250755   -4.21   
1252    JNJ 2020-10-20  144.66  145.75  144.12  144.55  5326540   -0.11   
1251    JNJ 2020-10-21  144.12  145.34  143.91  143.93  5221348   -0.19   
1250    JNJ 2020-10-22  144.16  145.69  143.84  145.08  5280253    0.92   

      changePercent      vwap  
1254        0.33875  147.8675  
1253       -2.83000  146.4700  
1252       -0.07604  144.7700  
1251       -0.13183  144.3250  
1250        0.63818  144.6925     symbol       date    open    high     low   close    volume  change  \
4    JNJ 2025-10-09  190.00  192.10  189.73  191.08   7484800    1.08   
3    JNJ 2025-10-10  192.01  192.07  189.85  190.72   9598019   -1.29   
2    JNJ 2025-10-13  189.83  191.30  189.28  190.90   7979500    1.07   
1    JNJ 2025-10-14  192.92  194.40  185.88  190.85  13613340   -

In [21]:
import requests
import pandas as pd
import plotly.express as px

def plot_normalized_price(ticker: str, api_key: str):
    # construye URL del endpoint “stable”
    url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={ticker}&apikey={api_key}"
    headers = {"User-Agent": "Mozilla/5.0"}

    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print("ERROR:", resp.status_code)
        print("Respuesta del servidor:", resp.text)
        raise RuntimeError(f"Error al conectar con FMP (stable) para {ticker}: {resp.status_code}")

    data = resp.json()
    hist = data["historical"] if "historical" in data else data

    df = pd.DataFrame(hist)
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")

    # Normalización min-max del precio de cierre
    min_price = df["close"].min()
    max_price = df["close"].max()
    df["close_norm"] = (df["close"] - min_price) / (max_price - min_price)

    fig = px.line(df, x="date", y="close_norm",
                  title=f"Precio de cierre normalizado 0-1 — {ticker}",
                  labels={"date": "Fecha", "close_norm": "Precio Normalizado"},
                  template="plotly_white")
    fig.show()

    print(ticker, " – primeros y últimos valores normalizados:")
    print(df[["date", "close", "close_norm"]].head())
    print(df[["date", "close", "close_norm"]].tail())

    return df


In [22]:
df_msft = plot_normalized_price("MSFT", API_KEY)
df_jnj = plot_normalized_price("JNJ", API_KEY)
df_aapl = plot_normalized_price("AAPL", API_KEY)


MSFT  – primeros y últimos valores normalizados:
           date   close  close_norm
1254 2020-10-16  219.66    0.051994
1253 2020-10-19  214.22    0.035672
1252 2020-10-20  214.65    0.036963
1251 2020-10-21  214.80    0.037413
1250 2020-10-22  214.89    0.037683
        date   close  close_norm
4 2025-10-09  522.40    0.960277
3 2025-10-10  510.96    0.925955
2 2025-10-13  514.05    0.935225
1 2025-10-14  513.57    0.933785
0 2025-10-15  513.43    0.933365


JNJ  – primeros y últimos valores normalizados:
           date   close  close_norm
1254 2020-10-16  148.10    0.203255
1253 2020-10-19  144.32    0.133346
1252 2020-10-20  144.55    0.137599
1251 2020-10-21  143.93    0.126133
1250 2020-10-22  145.08    0.147402
        date   close  close_norm
4 2025-10-09  191.08    0.998151
3 2025-10-10  190.72    0.991493
2 2025-10-13  190.90    0.994822
1 2025-10-14  190.85    0.993897
0 2025-10-15  191.18    1.000000


AAPL  – primeros y últimos valores normalizados:
           date   close  close_norm
1254 2020-10-16  119.02    0.068220
1253 2020-10-19  115.98    0.047987
1252 2020-10-20  117.51    0.058170
1251 2020-10-21  116.87    0.053910
1250 2020-10-22  115.75    0.046456
        date   close  close_norm
4 2025-10-09  254.04    0.966855
3 2025-10-10  245.27    0.908486
2 2025-10-13  247.66    0.924393
1 2025-10-14  247.77    0.925125
0 2025-10-15  249.34    0.935574
